In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,C,B,B,A,E,C,N,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,A,A,B,D,A,F,A,O,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,A,C,B,D,A,D,A,F,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,A,C,B,D,A,E,C,K,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,A,C,B,D,A,E,A,N,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [6]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()


,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,2.0,13.0,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,1.0,1.0,0.0,0.0,1.0,3.0,0.0,5.0,0.0,14.0,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,0.0,0.0,0.0,2.0,1.0,3.0,0.0,3.0,0.0,5.0,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,2.0,10.0,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,0.0,0.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


In [7]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()
'''

'\n# List of categorical columns\nobject_cols = [col for col in features.columns if \'cat\' in col]\n\n# one-hot-encode categorical columns\nX = features.copy()\nX_test = test.drop([\'id\'], axis=1).copy()\n\noh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")\nX_ohe = oh_encoder.fit_transform(features[object_cols])\nX_test_ohe = oh_encoder.transform(test[object_cols])\n\nX_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])\nX_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])\n\nX = pd.concat([X, X_ohe], axis=1)\nX_test = pd.concat([X_test, X_test_ohe], axis=1)\nX = X.drop(object_cols, axis=1)\nX_test = X_test.drop(object_cols, axis=1)\n    \n# Preview the one-hot-encoded features\nX.head()\n'

In [8]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [9]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Pseudo Label

In [10]:
pseudo = pd.read_csv("../input/30dml-pseudo2/xgb_submission_k.txt")[target]
train_pseudo = pd.concat([X, y], axis=1)
test_pseudo = pd.concat([X_test, pseudo], axis=1)
all_pseudo = pd.concat([train_pseudo, test_pseudo]).reset_index(drop=True)

# Optuna

In [11]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [12]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=all_pseudo.iloc[:,:-1], y=all_pseudo[target]):
  """
  """
  param_space = {
               #'device':'gpu',  # Use GPU acceleration
               #'boosting_type': 'gbdt',
               'reg_lambda':trial.suggest_uniform('reg_lambda', 29, 35),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 28, 34),
                'subsample': trial.suggest_uniform('subsample', 0.127, 0.227),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.0468,0.1268),
                #'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
               'learning_rate':trial.suggest_uniform('learning_rate', 2.47e-2, 4.47e-2),
                 'min_child_samples':trial.suggest_int('min_child_samples', 47, 59),
              'num_leaves':trial.suggest_int('num_leaves', 37, 53),
              'max_depth':trial.suggest_int('max_depth', 3, 7),
              #'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.005),
              #'class_weight':trial.suggest_categorical('class_weight',['balanced',None]),
               'n_estimators':N_ESTIMATORS,
                'n_jobs' : -1,
              'metric':'rmse',
              'max_bin':trial.suggest_int('max_bin', 777, 877),
              'cat_smooth':trial.suggest_int('cat_smooth', 16, 28),
              'cat_l2':trial.suggest_loguniform('cat_l2', 17.7, 29.7)
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  trn_idx = X_tr.index
  val_idx = X_va.index
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  oof_idx = np.array([idx for idx in val_idx if idx < train.shape[0]])
  preds_idx = np.array([idx for idx in val_idx if idx >= train.shape[0]])

  X_train, y_train = all_pseudo.iloc[trn_idx, :-1], all_pseudo[target].iloc[trn_idx]
  #X_train = apply_noise_row(X_train)

  X_valid, y_valid = all_pseudo.iloc[oof_idx, :-1], all_pseudo[target].iloc[oof_idx]
  #X_test = all_pseudo[scale_features].iloc[preds_idx]
  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = lgb.LGBMRegressor(**param_space)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
        categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
        #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
    )


    lgb_oof[oof_idx] += model.predict(X_valid) / len(seed_list)
    #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  #elapsed = time.time() - start
  rmse = mean_squared_error(y_valid, lgb_oof[oof_idx], squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [13]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-24 00:01:39,211] A new study created in memory with name: no-name-85adc3b1-2c5f-47b4-8dc4-0b1162f9c64e


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.712979
[2000]	valid_0's rmse: 0.710256
[3000]	valid_0's rmse: 0.709329
[4000]	valid_0's rmse: 0.709023
[5000]	valid_0's rmse: 0.708936
[6000]	valid_0's rmse: 0.708868
[7000]	valid_0's rmse: 0.708824
Early stopping, best iteration is:
[7231]	valid_0's rmse: 0.708815
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.713076
[2000]	valid_0's rmse: 0.710287
[3000]	valid_0's rmse: 0.709472
[4000]	valid_0's rmse: 0.709153
[5000]	valid_0's rmse: 0.709016
[6000]	valid_0's rmse: 0.708938
[7000]	valid_0's rmse: 0.708902
[8000]	valid_0's rmse: 0.708867
Early stopping, best iteration is:
[8745]	valid_0's rmse: 0.708849


[I 2021-08-24 00:06:34,155] Trial 0 finished with value: 0.7085765147166658 and parameters: {'reg_lambda': 32.958173239105804, 'reg_alpha': 33.36253373575549, 'subsample': 0.16499732885756266, 'colsample_bytree': 0.08557122940591796, 'learning_rate': 0.04240519644606573, 'min_child_samples': 52, 'num_leaves': 41, 'max_depth': 5, 'max_bin': 873, 'cat_smooth': 21, 'cat_l2': 26.566113015674667}. Best is trial 0 with value: 0.7085765147166658.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.713037
[2000]	valid_0's rmse: 0.710377
[3000]	valid_0's rmse: 0.70945
[4000]	valid_0's rmse: 0.709135
[5000]	valid_0's rmse: 0.709016
Early stopping, best iteration is:
[5626]	valid_0's rmse: 0.70895
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.71309
[2000]	valid_0's rmse: 0.710339
[3000]	valid_0's rmse: 0.709488
[4000]	valid_0's rmse: 0.709225
[5000]	valid_0's rmse: 0.709134
[6000]	valid_0's rmse: 0.709055
Early stopping, best iteration is:
[6689]	valid_0's rmse: 0.709032


[I 2021-08-24 00:10:28,593] Trial 1 finished with value: 0.7087332481758782 and parameters: {'reg_lambda': 30.815054668622405, 'reg_alpha': 31.600743869733975, 'subsample': 0.12735285757014822, 'colsample_bytree': 0.08040774730960737, 'learning_rate': 0.04230783781244815, 'min_child_samples': 57, 'num_leaves': 49, 'max_depth': 5, 'max_bin': 807, 'cat_smooth': 28, 'cat_l2': 19.794552863647922}. Best is trial 0 with value: 0.7085765147166658.


...

Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.715753
[2000]	valid_0's rmse: 0.712102
[3000]	valid_0's rmse: 0.710587
[4000]	valid_0's rmse: 0.709842
[5000]	valid_0's rmse: 0.709501
[6000]	valid_0's rmse: 0.709206
[7000]	valid_0's rmse: 0.709026
[8000]	valid_0's rmse: 0.70893
[9000]	valid_0's rmse: 0.708842
[10000]	valid_0's rmse: 0.708782
[11000]	valid_0's rmse: 0.708729
[12000]	valid_0's rmse: 0.708704
[13000]	valid_0's rmse: 0.708675
[14000]	valid_0's rmse: 0.708657
[15000]	valid_0's rmse: 0.708641
[16000]	valid_0's rmse: 0.708629
Early stopping, best iteration is:
[15873]	valid_0's rmse: 0.708626
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.716114
[2000]	valid_0's rmse: 0.712246
[3000]	valid_0's rmse: 0.710768
[4000]	valid_0's rmse: 0.709975
[5000]	valid_0's rmse: 0.709575
[6000]	valid_0's rmse: 0.709338
[7000]	valid_0's rmse: 0.709196
[8000]	valid_0's rmse: 0.709079
[9000]	valid_0's rmse: 0.709017
[1

[I 2021-08-24 02:28:51,380] Trial 28 finished with value: 0.7085074339931049 and parameters: {'reg_lambda': 33.93572114516547, 'reg_alpha': 33.62934849863741, 'subsample': 0.21953523935854952, 'colsample_bytree': 0.103283511215348, 'learning_rate': 0.03638976294584761, 'min_child_samples': 51, 'num_leaves': 41, 'max_depth': 4, 'max_bin': 870, 'cat_smooth': 23, 'cat_l2': 22.923539663380268}. Best is trial 28 with value: 0.7085074339931049.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.714562
[2000]	valid_0's rmse: 0.711376
[3000]	valid_0's rmse: 0.710115
[4000]	valid_0's rmse: 0.709519
[5000]	valid_0's rmse: 0.709232
[6000]	valid_0's rmse: 0.709042
[7000]	valid_0's rmse: 0.708919
[8000]	valid_0's rmse: 0.708852
[9000]	valid_0's rmse: 0.708785
[10000]	valid_0's rmse: 0.708744
Early stopping, best iteration is:
[10680]	valid_0's rmse: 0.708719
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.715033
[2000]	valid_0's rmse: 0.711622
[3000]	valid_0's rmse: 0.710286
[4000]	valid_0's rmse: 0.709719
[5000]	valid_0's rmse: 0.709391
[6000]	valid_0's rmse: 0.709195
[7000]	valid_0's rmse: 0.70908
[8000]	valid_0's rmse: 0.709016
[9000]	valid_0's rmse: 0.708967
[10000]	valid_0's rmse: 0.708912
[11000]	valid_0's rmse: 0.708881
[12000]	valid_0's rmse: 0.708862
[13000]	valid_0's rmse: 0.708839
[14000]	valid_0's rmse: 0.708826
Early stopping, best iteration is:


[I 2021-08-24 02:35:41,779] Trial 29 finished with value: 0.7085576390510198 and parameters: {'reg_lambda': 34.0188035399498, 'reg_alpha': 33.76374074749085, 'subsample': 0.21601062133127855, 'colsample_bytree': 0.08115813727344115, 'learning_rate': 0.04314077517210314, 'min_child_samples': 51, 'num_leaves': 40, 'max_depth': 4, 'max_bin': 870, 'cat_smooth': 23, 'cat_l2': 23.559253908877295}. Best is trial 28 with value: 0.7085074339931049.


Number of finished trials: 30
Best trial: {'reg_lambda': 33.93572114516547, 'reg_alpha': 33.62934849863741, 'subsample': 0.21953523935854952, 'colsample_bytree': 0.103283511215348, 'learning_rate': 0.03638976294584761, 'min_child_samples': 51, 'num_leaves': 41, 'max_depth': 4, 'max_bin': 870, 'cat_smooth': 23, 'cat_l2': 22.923539663380268}


In [14]:
study.best_params

{'reg_lambda': 33.93572114516547,
 'reg_alpha': 33.62934849863741,
 'subsample': 0.21953523935854952,
 'colsample_bytree': 0.103283511215348,
 'learning_rate': 0.03638976294584761,
 'min_child_samples': 51,
 'num_leaves': 41,
 'max_depth': 4,
 'max_bin': 870,
 'cat_smooth': 23,
 'cat_l2': 22.923539663380268}

# Log

====== Ordinal encoding =========

0.709167665584924 no noise ver9

0.7087162719562731 no noise ver16 (narrow space)

0.7086508785373714 no noise ver17 (narrow space) final

0.7127333982112984 row-wise noise ver10

0.7126600249836035 random noise ver11

====== One-hot encoding =========

0.7205030924594267 no noise ver13

0.722057702839371 random noise ver14

0.7222305639305339 row-wise noise ver15

==========================================
=========== Changed pseudo ===============

====== Ordinal encoding =========

0.709174903761106 no noise ver20

0.7086337003244899 no noise ver21 (narrow space)

0.708623028878119 no noise ver22 (narrow space)
